# TIKE LTS Kernel Testing
For the most part, I've tried to make it obvious where the packages are used by leaving the imports at the top of the cells.

**Last updated: Nov 2023**

## Astropy & Matplotlib

In [ ]:
import matplotlib.pyplot as plt

from astropy.time import Time
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization import astropy_mpl_style

# Set plotting style
plt.style.use(astropy_mpl_style)

# Check that we can set a time
times = ['1999-01-01T00:00:00.123456789', '2010-01-01T00:00:00']
t = Time(times, format='isot', scale='utc')

# Get an image, print info
image_file = get_pkg_data_filename('tutorials/FITS-images/HorseHead.fits')
fits.info(image_file)

# Get underlying data, print shape
image_data = fits.getdata(image_file, ext=0)
print(image_data.shape)

# Actually make/plot the figure
plt.figure()
plt.imshow(image_data, cmap='gray')
plt.colorbar()

## Astroquery & Boto3

In [ ]:
import s3fs

from astropy.io import fits
from astroquery.mast import Observations

Observations.enable_cloud_dataset()
fs = s3fs.S3FileSystem(anon=True)

obs = Observations.query_criteria(obs_collection="TESS", target_name='219796019', sequence_number=57)
if len(obs) == 0:
    raise Exception("Team Octarine: Contact MAST. Invalid target criteria")
prod = Observations.get_product_list(obs)
uris = Observations.get_cloud_uris(prod)

with fs.open(uris[0]) as f:
    fits.info(f)

## Batman
Also tests numpy and matplotlib

In [ ]:
import batman
import numpy as np
import matplotlib.pyplot as plt

params = batman.TransitParams()       #object to store transit parameters
params.t0 = 0.                        #time of inferior conjunction
params.per = 1.                       #orbital period
params.rp = 0.1                       #planet radius (in units of stellar radii)
params.a = 15.                        #semi-major axis (in units of stellar radii)
params.inc = 87.                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 90.                        #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5, 0.1, 0.1, -0.1]      #limb darkening coefficients [u1, u2, u3, u4]

t = np.linspace(-0.025, 0.025, 1000)  #times at which to calculate light curve
m = batman.TransitModel(params, t)    #initializes model

flux = m.light_curve(params)    
radii = np.linspace(0.09, 0.11, 20)
for r in radii:
        params.rp = r                           #updates planet radius
        new_flux = m.light_curve(params)        #recalculates light curve
        plt.plot(t, new_flux)

## Bokeh

In [ ]:
from bokeh.io import output_notebook

output_notebook()

''' A color mapping plot with color spectrum scale. The example plots demonstrates
log mapping and linear mapping with different color palette.

.. bokeh-example-metadata::
    :apis: bokeh.plotting.Figure.scatter, bokeh.models.ColumnDataSource, bokeh.models.annotations.ColorBar, bokeh.models.mappers.LinearColorMapper, bokeh.models.mappers.LogColorMapper
    :refs: :ref:`ug_topics_images_colormapped`
    :keywords: color, tools, scatter, data_map

''' # noqa: E501
import numpy as np

from bokeh.layouts import column, gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap, log_cmap

x = np.random.random(size=2000) * 1000
y = np.random.normal(size=2000) * 2 + 5
source = ColumnDataSource(dict(x=x, y=y))

def make_plot(mapper, palette):
    cmap = mapper("x", palette=palette, low=1, high=1000)
    axis_type = mapper.__name__.split("_")[0] # linear or log

    p = figure(x_range=(1, 1000), title=f"{palette} with {mapper.__name__}",
               toolbar_location=None, tools="", x_axis_type=axis_type)

    r = p.scatter('x', 'y', alpha=0.8, source=source, color=cmap)

    color_bar = r.construct_color_bar(padding=0,
                                      ticker=p.xaxis.ticker,
                                      formatter=p.xaxis.formatter)
    p.add_layout(color_bar, 'below')

    return p

p1 = make_plot(linear_cmap, "Viridis256")
p2 = make_plot(log_cmap, "Viridis256")
p3 = make_plot(linear_cmap, "Viridis6")
p4 = make_plot(log_cmap, "Viridis6")

p5 = figure(x_range=(1, 1000), width=800, height=300, toolbar_location=None, tools="",
            title="Viridis256 with linear_cmap, low/high = 200/800 = pink/grey")
cmap = linear_cmap("x", palette="Viridis256", low=200, high=800,
                   low_color="pink", high_color="darkgrey")
p5.scatter(x='x', y='y', alpha=0.8, source=source, color=cmap)

grid =  gridplot([[p1, p2], [p3, p4]], width=400, height=300, toolbar_location=None)
show(column(grid, p5))

## Dask

In [ ]:
import dask

df = dask.datasets.timeseries(freq='1d')
df.dtypes
%matplotlib inline
df[['x', 'y']].resample('24h').mean().compute().plot();

## Juliet

In [ ]:
import juliet
import matplotlib.pyplot as plt

# First, get arrays of times, normalized-fluxes and errors for HATS-46
# from Sector 1 from MAST:
t,f,ferr  = juliet.get_TESS_data('https://archive.stsci.edu/hlsps/tess-data-alerts/'+\
                                 'hlsp_tess-data-alerts_tess_phot_00403224672-'+\
                                 's01_tess_v1_lc.fits')

# Create dictionaries:
times, fluxes, fluxes_error = {},{},{}
# Save data into those dictionaries:
times['TESS'], fluxes['TESS'], fluxes_error['TESS'] = t,f,ferr


priors = {}

# Name of the parameters to be fit:
params = ['P_p1','t0_p1','r1_p1','r2_p1','q1_TESS','q2_TESS','ecc_p1','omega_p1',\
              'rho', 'mdilution_TESS', 'mflux_TESS', 'sigma_w_TESS']

# Distribution for each of the parameters:
dists = ['normal','normal','uniform','uniform','uniform','uniform','fixed','fixed',\
                 'loguniform', 'fixed', 'normal', 'loguniform']

# Hyperparameters of the distributions (mean and standard-deviation for normal
# distributions, lower and upper limits for uniform and loguniform distributions, and
# fixed values for fixed "distributions", which assume the parameter is fixed)
hyperps = [[1.,0.1], [1325.55,0.1], [0.,1], [0.,1.], [0., 1.], [0., 1.], 0.0, 90.,\
                   [100., 10000.], 1.0, [0.,0.1], [0.1, 1000.]]

# Populate the priors dictionary:
for param, dist, hyperp in zip(params, dists, hyperps):
    priors[param] = {}
    priors[param]['distribution'], priors[param]['hyperparameters'] = dist, hyperp

# Load dataset into juliet, save results to a temporary folder called toi141_fit:
dataset = juliet.load(priors=priors, t_lc = times, y_lc = fluxes, \
                      yerr_lc = fluxes_error, out_folder = 'toi141_fit')

# Fit and absorb results into a juliet.fit object:
# SHOULD TAKE ~10 seconds
results = dataset.fit(n_live_points = 10, nwalkers=2)

In [2]:
import juliet
juliet.load?

Init signature:
juliet.load(
    priors=None,
    starting_point=None,
    input_folder=None,
    t_lc=None,
    y_lc=None,
    yerr_lc=None,
    t_rv=None,
    y_rv=None,
    yerr_rv=None,
    GP_regressors_lc=None,
    linear_regressors_lc=None,
    GP_regressors_rv=None,
    linear_regressors_rv=None,
    out_folder=None,
    lcfilename=None,
    rvfilename=None,
    GPlceparamfile=None,
    GPrveparamfile=None,
    LMlceparamfile=None,
    LMrveparamfile=None,
    lctimedef='TDB',
    rvtimedef='UTC',
    ld_laws='quadratic',
    priorfile=None,
    lc_n_supersamp=None,
    lc_exptime_supersamp=None,
    lc_instrument_supersamp=None,
    mag_to_flux=True,
    verbose=False,
    matern_eps=0.01,
    george_hodlr=True,
    pickle_encoding=None,
)
Docstring:     
Given a dictionary with priors (or a filename pointing to a prior file) and data either given through arrays
or through files containing the data, this class loads data into a juliet object which holds all the information
abo

In [ ]:
import juliet
priors = {}

# Name of the parameters to be fit:
params = ['P_p1','t0_p1','mu_CORALIE14', \
          'mu_CORALIE07','mu_HARPS','mu_FEROS',\
          'K_p1', 'ecc_p1', 'omega_p1', 'sigma_w_CORALIE14','sigma_w_CORALIE07',\
           'sigma_w_HARPS','sigma_w_FEROS']

# Distributions:
dists = ['normal','normal','uniform', \
         'uniform','uniform','uniform',\
         'uniform','fixed', 'fixed', 'loguniform', 'loguniform',\
         'loguniform', 'loguniform']

# Hyperparameters
hyperps = [[1.007917,0.000073], [2458325.5386,0.0011], [-100,100], \
           [-100,100], [-100,100], [-100,100], \
           [0.,100.], 0., 90., [1e-3, 100.], [1e-3, 100.], \
           [1e-3, 100.], [1e-3, 100.]]

# Populate the priors dictionary:
for param, dist, hyperp in zip(params, dists, hyperps):
    priors[param] = {}
    priors[param]['distribution'], priors[param]['hyperparameters'] = dist, hyperp

dataset = juliet.load(priors = priors, rvfilename='rvs_toi141.dat', out_folder = 'toi141_rvs')
results = dataset.fit(n_live_points = 10, nwalkers=2)

## Lightkurve

These tests will take about 30 seconds to run

In [ ]:
import lightkurve as lk
%matplotlib inline
lc = lk.search_lightcurve('KIC 6185476', author='Kepler', cadence='long').download_all().stitch()
lc.plot();

clc = lc.flatten(21)
clc.plot();

p, t0 = 17.660114, 136.57258
folded_lc = clc.fold(period=p, epoch_time=t0)
folded_lc.scatter();

folded_lc.plot_river(bin_points=1, method='sigma');

## Scikit-learn

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.cluster import SpectralCoclustering
from sklearn.datasets import make_biclusters
from sklearn.metrics import consensus_score

data, rows, columns = make_biclusters(
    shape=(300, 300), n_clusters=5, noise=5, shuffle=False, random_state=0
)

plt.matshow(data, cmap=plt.cm.Blues)
plt.title("Original dataset")

# shuffle clusters
rng = np.random.RandomState(0)
row_idx = rng.permutation(data.shape[0])
col_idx = rng.permutation(data.shape[1])
data = data[row_idx][:, col_idx]

plt.matshow(data, cmap=plt.cm.Blues)
plt.title("Shuffled dataset")

model = SpectralCoclustering(n_clusters=5, random_state=0)
model.fit(data)
score = consensus_score(model.biclusters_, (rows[:, row_idx], columns[:, col_idx]))

print("consensus score: {:.3f}".format(score))

fit_data = data[np.argsort(model.row_labels_)]
fit_data = fit_data[:, np.argsort(model.column_labels_)]

plt.matshow(fit_data, cmap=plt.cm.Blues)
plt.title("After biclustering; rearranged to show biclusters")

plt.show()

## Scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d, RectBivariateSpline

x = np.arange(-5.01, 5.01, 0.25)
y = np.arange(-5.01, 7.51, 0.25)
xx, yy = np.meshgrid(x, y)
z = np.sin(xx**2 + 2.*yy**2)
f = interp2d(x, y, z, kind='cubic')

def plot(f, xnew, ynew):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
    znew = f(xnew, ynew)

    ax1.plot(x, z[0, :], 'ro-', xnew, znew[0, :], 'b-')

    im = ax2.imshow(znew)
    plt.colorbar(im, ax=ax2)

    plt.show()
    return znew

xnew = np.arange(-5.01, 5.01, 1e-2)
ynew = np.arange(-5.01, 7.51, 1e-2)
znew_i = plot(f, xnew, ynew)

## TESS_Point

In [ ]:
from tess_stars2px import tess_stars2px_function_entry

ra = 84.291188 
dec = -80.469119
ticid = 261136679

outID, outEclipLong, outEclipLat, outSec, outCam, outCcd, \
        outColPix, outRowPix, scinfo = tess_stars2px_function_entry(
                ticid, ra, dec)
for i in range(len(outID)):
    print('{0:d} {1:d} {2:d} {3:d} {4:f} {5:f}'.format(outID[i], outSec[i], \
      outCam[i], outCcd[i], outColPix[i], outRowPix[i]))
    
# For efficiency purposes if you save scinfo between calls
#  you will save time in setting up the the telescope fields
outID, outEclipLong, outEclipLat, outSec, outCam, outCcd, \
        outColPix, outRowPix, scinfo = tess_stars2px_function_entry(
                ticid, ra, dec, scInfo=scinfo)
print('Faster to re-use scinfo in repeated calls')
for i in range(len(outID)):
    print('{0:d} {1:d} {2:d} {3:d} {4:f} {5:f}'.format(outID[i], outSec[i], \
      outCam[i], outCcd[i], outColPix[i], outRowPix[i]))

print('Also accepts multiple targets')
ra = np.array([219.90085,10.897379])
dec = np.array([-60.835619,-17.986606])
ticid = np.array([0,1])
outID, outEclipLong, outEclipLat, outSec, outCam, outCcd, \
        outColPix, outRowPix, scinfo = tess_stars2px_function_entry(
                ticid, ra, dec, scInfo=scinfo)
for i in range(len(outID)):
    print('{0:d} {1:d} {2:d} {3:d} {4:f} {5:f}'.format(outID[i], outSec[i], \
          outCam[i], outCcd[i], outColPix[i], outRowPix[i]))

## Pandas

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "Name": [
            "Braund, Mr. Owen Harris",
            "Allen, Mr. William Henry",
            "Bonnell, Miss. Elizabeth",
        ],
        "Age": [22, 35, 58],
        "Sex": ["male", "male", "female"],
    }
)

df["Age"].max()

## Requests

In [ ]:
import requests
r = requests.get('https://httpbin.org/basic-auth/user/pass', auth=('user', 'pass'))
print(r.status_code, r.headers['content-type'], r.encoding, r.text, r.json())